In [1]:
### Делаем названия книг Дарьи Донцовой

In [2]:
import re
from bs4 import BeautifulSoup
from gensim.models import KeyedVectors, Word2Vec  

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [3]:
RUSSIAN = re.compile(r"[А-ЯЁа-яё \-]+")

In [4]:
# эмбеддинги
vectors = KeyedVectors.load_word2vec_format("./skipgram_vec.gz")

In [5]:
# загружаем список фразеологизмов, собираем подходящие, записываем

In [6]:
phrases_page = ""
with open("./rus_phrases.html") as f:
    for line in f.readlines():
        phrases_page += line + "\n"

In [7]:
soup = BeautifulSoup(phrases_page)

In [8]:
found_parts = soup.find_all("span", class_ = "html-attribute-value")

In [9]:
phrases = []
for part in found_parts:
    part = part.get_text().replace("(страница не существует)", "")
    if re.fullmatch(RUSSIAN, part.strip()):
        if len(part.split()) in range(3,5):
            phrases.append(part)

In [10]:
len(phrases)

631

In [11]:
# достаём названия книг на таких же условиях

In [12]:
with open("rus_books", "r") as f:
    for line in f.readlines():
        if "-" in line:
            part = line.split("-")[-1].strip()
            if len(part.split()) in range(3,5):
                phrases.append(part.lower())

In [13]:
len(phrases)

661

In [14]:
# убираем из фраз глагольные конструкции, чтобы 

In [15]:
phrases_without_infs = []

for phrase in phrases:
    words = phrase.split()
    counter = 0
    for word in words:
        if "INFN" in morph.parse(word)[0].tag or word == "как":
            counter +=1
    if not counter:
        phrases_without_infs.append(phrase)

In [16]:
len(phrases_without_infs)

326

In [17]:
# предобработка и собственно генерация названия

In [18]:
def is_noun_nominative(word: str) -> bool:
    """
    проверяем, существительное ли перед нами
    """
    if ("NOUN" in morph.parse(word)[0].tag) and (morph.parse(word)[0].normal_form == word):
        return True
    return False

In [19]:
def get_gender(word: str): 
    """
    определяем род
    """
    p = morph.parse(word)[0]
    return p.tag.gender

In [20]:
def get_dontsova_booktitle(noun):
    """
    поехали
    """
    closest_phrase = "такого слова не нашлось в корпусе :("  
    new_phrases = [x.split() for x in phrases_without_infs]
    closest_distance = 0
    closest_word = "PAD" 
    for i, phrase in enumerate(new_phrases):
        for word in phrase:
            if is_noun_nominative(word): 
                try:
                    distance = vectors.similarity(word+"_NOUN", noun+"_NOUN")
                    if (distance > closest_distance) and word != noun and \
                    get_gender(word) == get_gender(noun): 
                        closest_phrase = phrases_without_infs[i]
                        closest_word = word
                        closest_distance = distance 
                except KeyError:
                    continue  
                    
    return closest_phrase.replace(closest_word, noun).strip()

In [21]:
get_dontsova_booktitle("продюсер")

'отставной козы продюсер'

In [22]:
get_dontsova_booktitle("сникерс")

'кнут и сникерс'

In [23]:
get_dontsova_booktitle("чикибамбони")

'такого слова не нашлось в корпусе :('

In [24]:
get_dontsova_booktitle("телепузик")

'телепузик на побегушках'

In [25]:
get_dontsova_booktitle("мартышка")

'где мартышка зарыта'

In [26]:
get_dontsova_booktitle("дискета")

'дискета всё стерпит'

In [27]:
### классные ошибки

In [28]:
get_dontsova_booktitle("ледокол")

'с лёгким ледокол'